### I am importing random for random number to determine if someone gets sick or not

In [5]:
import random

### let's design our Person class with attributes listed above and methods listed above
Person Object

In [6]:
class Person(object):
    def __init__(self, default_days_sick=5, default_infection_rate=10, masked=False):
        self.ID = 0
        self.age = 0
        self.status = "Susceptible"
        self.days_sick = 0
        self.max_days_sick = default_days_sick
        self.infection_rate = default_infection_rate
        self.interactions = []
        self.is_masked = masked
        
        if (self.is_masked):
            self.infection_rate = self.infection_rate/2
        
    def set_is_masked(self, masked):
        self.is_masked = masked
        if (self.is_masked):
            self.infection_rate = self.infection_rate/2
    
    def get_is_masked(self):
        return self.is_masked
        
    def set_infection_rate(self, new_rate):
        self.infection_rate = new_rate
    
    def infect(self, number_of_days):
        self.days_sick = number_of_days
        self.status = "Infected"
        
    def recover(self):
        self.days_sick = 0
        self.status = "Recovered"
        
    def roll_for_infection(self):
        if (random.randint(0,101) < self.infection_rate):
            self.infect(self.max_days_sick)
            
    def get_status(self):
        return self.status
    
    def get_days_sick(self):
        return self.days_sick
    
    def update(self):
        if (self.status == "Susceptible"):
            self.roll_for_infection()
            
        elif (self.status == "Infected"):
            self.days_sick = self.days_sick - 1
            if (self.days_sick <= 0):
                self.recover()
                
                
    
    

In [7]:
Joe = Person()
print ("Joe is currently:", Joe.get_status(), "Joe has been sick for", Joe.get_days_sick(), " days")

Joe is currently: Susceptible Joe has been sick for 0  days


In [ ]:
Joe.infect(30)

In [ ]:
print ("Joe is currently:", Joe.get_status(), ", Joe will be sick for", Joe.get_days_sick(), " days")

In [ ]:
day = 0
Joe = Person()

while (Joe.get_status() != "Recovered"):
    day = day + 1
    Joe.update()
    print ("Day", day)
    print ("Joe is currently:", Joe.get_status(), ", Joe will be sick for", Joe.get_days_sick(), " days")

In [ ]:
day = 0
run = 0
Joe = Person()
Jane = Person()

while (Jane.get_status() != "Recovered"):
    day = day + 1
    Joe.update()

    ## Jane only gets updated when Joe is Infected or when Jane gets infected
    ## (Jane can only get infected when Joe *is* infected)
    ## (When Jane is infected, we have to update her, or she'll never Recover)
    if (Joe.get_status() == "Infected" or Jane.get_status() == "Infected"):
        Jane.update()
    
    ## If Joe is Recovered, and Jane is still Susceptible, then reset Joe
    
    if (Joe.get_status() == "Recovered" and Jane.get_status() == "Susceptible"):
        Joe = Person()
        run = run + 1
        print("------ RESET ------  run::", run)
        
    print ("Day", day)
    print ("Joe is currently :", Joe.get_status(), ", Joe will be sick for", Joe.get_days_sick(), " days")
    print ("Jane is currently:", Jane.get_status(), ", Jane will be sick for", Jane.get_days_sick(), " days")
        
    
    
    

### Something is still not quite right, the code works
### but there's still too much going on
---

#### We need a Population Class

## Population Object

In [21]:
class Population(object):
    def __init__(self, number_of_people=1000, default_interactions=10, default_days_sick=5, default_infection_rate=10, default_percentage_masked=0):
        self.People = []
        self.population_count = number_of_people
        self.number_of_interactions = default_interactions
        self.max_days_sick = default_days_sick
        self.infection_rate = default_infection_rate
        
        self.number_masked = number_of_people*(default_percentage_masked/100)
        
        ## set Patient_Zero aka The Jerk
        patient_zero = Person()
        patient_zero.infect(self.max_days_sick)
        
        ## add patient_zero to our People[]
        
        self.People.append(patient_zero)
        
        ## build the rest of our Population      
        for i in range(0, self.population_count):
            new_person = Person()
            if (i <= self.number_masked):
                new_person.set_is_masked(True)
                
            self.People.append(new_person)
            
    #return mumber of people masked
    def get_number_masked(self):
        return self.number_masked;
    
    def get_number_masked_infected(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Infected" and the_person.get_is_masked() == True):
                count = count + 1
        
        return count    
    
    ##go through our population, and count each person that's susceptible
    def number_of_susceptible(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Susceptible"):
                count = count + 1
        
        return count
    
    ##go through the population, and count each person that's infected
    def number_of_infected(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Infected"):
                count = count + 1
        
        return count
    
    ##go through the population, and count each person that's recovered
    def number_of_recovered(self):
        count = 0
        
        for the_person in self.People:
            if (the_person.get_status() == "Recovered"):
                count = count + 1
                
        return count
    
    ##Total Population
    def get_population_count(self):
        return len(self.People)
    
    
    def update(self):
        
        ##we're going to make a list of People who might get
        ##infected at the end of the day
        
        might_get_sick = []
        
        ## go through my Population and see who might get sick
        for person in self.People:
            person.interactions = []
            
            ## each person inteteracts w/ number_of_interaction number of people
            
            for i in range(0, self.number_of_interactions):
                r = random.randint(0, self.population_count)
                person.interactions.append(r)
            
            ## if person is infected, their interactions might get infected
            if (person.get_status() == "Infected"):
                ##????? really????
                person.update()
                
                for j in person.interactions:
                    if (self.People[j].get_status() == "Susceptible"):
                        might_get_sick.append(self.People[j])
            
            ## if person is susceptible, they interact w/ someone infected
            if (person.get_status() == "Susceptible"):
                for j in person.interactions:
                    if (self.People[j].get_status() == "Infected"):
                        might_get_sick.append(person)
                        
        ##the above updates each person who is infected
        ##I have a list of people who **might** get infected
        ##each person in that list, has to roll to get infected
        
        for person in might_get_sick:
            person.roll_for_infection()
            
            
    def get_population_list(self):
        return self.People
    

### Our Main Code!
 Community Object 

In [24]:
##Build our Community

myCommunity = Population(40000)
day = 0
number_infected = myCommunity.number_of_infected()
while (number_infected > 0):
    number_infected = myCommunity.number_of_infected()
    number_recovered = myCommunity.number_of_recovered()
    print("day:", day, "infected:", "recovered:", number_recovered)
    
    day = day + 1
    myCommunity.update()



day: 0 infected: recovered: 0
day: 1 infected: recovered: 0
day: 2 infected: recovered: 0
day: 3 infected: recovered: 0
day: 4 infected: recovered: 0
day: 5 infected: recovered: 1
day: 6 infected: recovered: 4
day: 7 infected: recovered: 12
day: 8 infected: recovered: 29
day: 9 infected: recovered: 90
day: 10 infected: recovered: 270
day: 11 infected: recovered: 796
day: 12 infected: recovered: 2329
day: 13 infected: recovered: 6369
day: 14 infected: recovered: 15413
day: 15 infected: recovered: 28580
day: 16 infected: recovered: 37288
day: 17 infected: recovered: 39568
day: 18 infected: recovered: 39924
day: 19 infected: recovered: 39984
day: 20 infected: recovered: 39995
day: 21 infected: recovered: 39997
day: 22 infected: recovered: 39998


In [26]:
##Build our Community

myCommunity = Population(40000, 10, 5, 10, 70)
print ("Population:",myCommunity.get_population_count())
print ("Number masked:", myCommunity.get_number_masked())


day = 0
number_infected = myCommunity.number_of_infected()
while (number_infected > 0):
    number_infected = myCommunity.number_of_infected()
    number_recovered = myCommunity.number_of_recovered()
    number_masked = myCommunity.get_number_masked_infected()
    
    print("day:", day, "infected:", number_infected, "masked+infected:", number_masked, "recovered:", number_recovered)
    
    day = day + 1
    myCommunity.update()

Population: 40001
Number masked: 28000.0
day: 0 infected: 1 masked+infected: 0 recovered: 0
day: 1 infected: 2 masked+infected: 0 recovered: 0
day: 2 infected: 4 masked+infected: 2 recovered: 0
day: 3 infected: 8 masked+infected: 5 recovered: 0
day: 4 infected: 22 masked+infected: 13 recovered: 0
day: 5 infected: 56 masked+infected: 31 recovered: 1
day: 6 infected: 125 masked+infected: 64 recovered: 2
day: 7 infected: 263 masked+infected: 142 recovered: 4
day: 8 infected: 574 masked+infected: 308 recovered: 8
day: 9 infected: 1246 masked+infected: 668 recovered: 22
day: 10 infected: 2725 masked+infected: 1492 recovered: 57
day: 11 infected: 5688 masked+infected: 3147 recovered: 127
day: 12 infected: 10991 masked+infected: 6244 recovered: 267
day: 13 infected: 18700 masked+infected: 11178 recovered: 582
day: 14 infected: 26639 masked+infected: 16959 recovered: 1268
day: 15 infected: 31344 masked+infected: 21071 recovered: 2782
day: 16 infected: 31558 masked+infected: 22278 recovered: 58